# 5. PDF-linkityskonversio

## 5.1 PDF-linkittäjä

PDF-liitteiden linkittäminen yhtenäisessä muodossa kaavojen paikkatietoihin on mahdollista, jos tiedetään, mihin kaavaindeksitunnukseen kukin liite kytkeytyy. VOOKA-pilottiprojektissa tämä tieto puuttui lähes poikkeuksetta kaavojen paikkatietoaineistoista. Tästä syystä kaavaliitteiden ja niitä vastaavien kaavaindeksien tunnistustyö toteutettiin manuaalisena työnä.

Työssä PDF-linkitystä varten koostettiin liitostaulu, jossa jokaisella rivillä on tieto kaavan indeksitunnuksesta (KTJ, kunnan aineisto tai molemmat) sekä kaavan dokumenttityyppi (esim. osallistamis- ja arviointisuunnitelma). Kaiken kaikkiaan liitostaulun skeema oli seuraavanlainen:

| Ominaisuustieto | Selite |
| :--- | :--- |
| Kunta | Kunnan virallinen kuntakoodi |
| Kunnan indeksitunnus | Kunnan paikkatietomuotoisen kaava-aineiston kaavaindeksitunnus |
| KTJ-indeksitunnus | KTJ-aineiston kaavaindeksitunnus ("kaavatunnus_1") |
| Original filename | Kunnalta saadun kaavaliitteen alkuperäinen tiedostonimi |
| Kaavalaji | Kaavalajikoodisto ylätasolla |
| Dokumentin tyyppi | Dokumenttityyppikoodisto, sanallinen |
| Dokumentin tyyppi2 | Dokumenttityyppikoodisto, numeerinen |
| Tila | Koodisto, joka kertoo, onko kaavaliite validi |
| Voimassa oleva | Onko liite voimassa vai ei (boolean) |
| Multipage | Onko liitteessä useampi kuin yksi sivu (boolean) |

Kaavalaji-koodisto:
- ak
- rak
- yk

Dokumentin tyyppi -koodisto (sanallinen ja numeerinen):
- 1 = kaavakartta (sis. merkinnät ja määräykset)
- 2 = kaavakartta (ei sis. merkinnät ja määräykset)
- 3 = merkinnät ja määräykset (erillisenä)
- 4 = kaavaselostus
- 5 = osallistamis- ja arviointisuunnitelma
- 6 = muu

Tila-koodisto:
- ok
- ei ok

<span style="color:red">Kesken, täydennetään!</span>

## 5.2 PDF/A-konversio

ETL-työkalu konvertoi kunnilta saadut liite-PDF-tiedostot automatisoidusti Ghostscript-muuntimella PDF/A-arkistoformaattiin. Muunnin perustuu soveltuvin osin GitHubista avoimesti saatavilla olevaan [Python-Ghostscript-konvertteriin](https://github.com/Rajasekaran85/Python-PDF-A-Conversion). VOOKA-projektin tarpeisiin muokattu muunnin löytyy omana versionaan projektin [GitHub-sivuilta](https://github.com/ubigu/vooka).

Jos saatu PDF-formaatti on toteutettu versiolla, jota ei saada PDF/A-muotoon, se tallennetaan sellaisenaan. Mikäli jotkin kunnan PDF-tiedostot eivät käänny kunnan aineistosta johtuvista teknisistä syistä PDF/A-muotoon, tallennetaan nämä normaalissa PDF-muodossa.

ETL-työkalun PDF/A-konvertteri kysyy käyttäjältä tiedostopolkuja input- ja output-kansioihin. Muuntaja kääntää kaikki input-kansiossa olevat PDF-tiedostot PDF/A-formaattiin ja tallentaa ne output-kansioon.

In [ ]:
from lib.pdf_a_conv import writePdfA
writePdfA()